In [1]:
#import re  # Для препроцессинга
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
# Загружаю данные
df_clean = pd.read_csv(r'PRE.csv')

In [3]:
df_clean.head(-5)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр отдельный л...
2,первомайский мина пешкомт плая уютный квартира...
3,шаговый доступность парк сокольники развитой и...
4,ильинский коммуникация центральный квартира хо...
...,...
100713,переуступка входить цена корпус комната изолир...
100714,продаваться трехкомнатная квартира замечательн...
100715,прочитайте объявление полностью звонить строго...
100716,читать звоним согласно статья фз исполнительны...


In [4]:
#использую реализацию Gensim Doc2Vec
import multiprocessing
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [5]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [6]:
df_clean['clean']=df_clean['clean'].fillna("Без описания")# меняю нан на "Без описания"

In [7]:
#токенизация и добавление в текст
description = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df_clean.clean)]

In [8]:
#размер вектора, количество измерений, количество переборов, количество ядер
d2v_model = Doc2Vec(vector_size=64, min_count=1, epochs = 20, workers=cores)

INFO - 23:23:52: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w5,s0.001,t16)', 'datetime': '2022-11-30T23:23:52.403981', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [9]:
#Модель
d2v_model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=cores, epochs = 20)
#Словарь
d2v_model.build_vocab(description)
#Обучение модели
d2v_model.train(description, total_examples=d2v_model.corpus_count
            , epochs=d2v_model.epochs)

INFO - 23:23:55: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w2,s0.001,t16)', 'datetime': '2022-11-30T23:23:55.256766', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 23:23:55: collecting all words and their counts
INFO - 23:23:55: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 23:23:55: PROGRESS: at example #10000, processed 633411 words (7272594/s), 14572 word types, 0 tags
INFO - 23:23:55: PROGRESS: at example #20000, processed 1224343 words (7535917/s), 19483 word types, 0 tags
INFO - 23:23:55: PROGRESS: at example #30000, processed 1776316 words (7228088/s), 21530 word types, 0 tags
INFO - 23:23:55: PROGRESS: at example #40000, processed 2331132 words (7119789/s), 23159 word types, 0 tags
INFO - 23:23:55: PROGRESS: at example #50000, processed 3260837 words (8027590/s), 25968 word types, 0 tags
INFO - 23:23:55: PROGRESS: a

In [10]:
#Генерация векторов
des2vec = [d2v_model.infer_vector((df_clean['clean'][i].split(' '))) for i in range(0,len(df_clean['clean']))]
des2vec

[array([ 0.24193111, -0.11010161,  0.06741308,  0.03081798, -0.12203565,
        -0.10634431,  0.3757462 , -0.1580089 ,  0.14898916,  0.20470835,
        -0.07053038,  0.26698884, -0.17632112,  0.13250914,  0.25547817,
        -0.01706447,  0.14804599,  0.17511262,  0.1653814 , -0.4146311 ,
         0.06634974, -0.11842316,  0.15547583,  0.1182752 ,  0.26578775,
         0.0766857 , -0.13871644, -0.16174833,  0.08932503, -0.02611912,
        -0.07832705, -0.0572519 , -0.19804803, -0.26993233, -0.27130717,
        -0.04092547, -0.02471383, -0.05209883, -0.09960645,  0.36870688,
        -0.20717843,  0.04739678,  0.6139252 , -0.09081388, -0.10053486,
         0.26575637, -0.05884986, -0.43127823,  0.16868263,  0.36764482,
        -0.06336853, -0.11287676, -0.04164437, -0.26168334, -0.00669549,
        -0.23423308, -0.14787424,  0.14027952, -0.11339553,  0.03435834,
        -0.17553347,  0.07676079, -0.20218587, -0.1585974 ], dtype=float32),
 array([ 0.21361424,  0.21263786,  0.24576573, 

In [11]:
#Проверяю тип данных
des2vec[0].dtype


dtype('float32')

In [12]:
#Переписываю векторы в датафрем
Vec64 = pd.DataFrame(data = des2vec)

In [13]:
Vec64.head(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.241931,-0.110102,0.067413,0.030818,-0.122036,-0.106344,0.375746,-0.158009,0.148989,0.204708,...,-0.006695,-0.234233,-0.147874,0.140280,-0.113396,0.034358,-0.175533,0.076761,-0.202186,-0.158597
1,0.213614,0.212638,0.245766,0.143428,0.120143,-0.357147,0.020020,-0.193174,0.105130,-0.264720,...,0.185037,0.159005,0.327853,0.030575,-0.258869,0.073926,-0.354773,-0.086634,-0.083417,-0.027566


In [14]:
Vec64.shape

(100723, 64)

In [ ]:
#import numpy as np
#Создание списка списков
#добавление списка в столбец данных
#df_clean['doc2vec'] = np.array(des2vec).tolist()

#Это не обязательное условие
#df_clean= df_clean.drop([0],axis=1)

#df_clean.head(-5)

In [15]:
#считываю csv для объединеня
df1 = pd.read_csv(r'Сlean.csv')

In [23]:
#конкатенация
Data = pd.concat([df1, Vec64], axis=1)
Data=Data.drop(['description'], axis=1)#Удаляю столбец с описанием
Data.head(2)

,id,rooms,cost(₽),square(м²),cost_for_meter(₽),address,metro_name,distance(м),center_distance/m,floor,...,54,55,56,57,58,59,60,61,62,63
0,0,1 – комнатная,5000000.0,31.0,161290.3,"Центральная ул., 6",бульвар дмитрия донского,4995.0,20600.0,2,...,-0.006695,-0.234233,-0.147874,0.140280,-0.113396,0.034358,-0.175533,0.076761,-0.202186,-0.158597
1,1,1 комната,4500000.0,98.0,45918.4,"Коптевская ул., 28к2",коптево,420.0,11400.0,3,...,0.185037,0.159005,0.327853,0.030575,-0.258869,0.073926,-0.354773,-0.086634,-0.083417,-0.027566


In [24]:
#Сохраняю данные
Data.to_csv(r'Data1.csv',index=False)